In [22]:
from multiprocessing import Manager
from multiprocessing.pool import Pool
import numpy as np
manager = Manager()
l1 = manager.list()
l2 = manager.list()
pool = Pool(None)
ll = []
for i in range(100):
    # pool.apply(func=l1.append, args=(i,))
    ll.append(i)

pool.starmap(func=l1.append, iterable=ll)
pool.starmap(func=l2.append, iterable=ll)

pool.close()
pool.join()
cnt1=0
cnt2=0
for a in np.arange(100):
    if(l1[a]==a):
        cnt1+=1
    if(l2[a]==a):
        cnt2+=1
    
print(cnt1, cnt2)



TypeError: 'int' object is not iterable

In [ ]:
import multiprocessing
def f(l):
		# 在子进程中向共享列表中添加元素
		l.append(multiprocessing.current_process().name)
if name == 'main':
		# 创建一个Manager对象
		manager = multiprocessing.Manager()
		# 创建一个可在多个进程之间共享的列表对象
		l = manager.list()
		# 创建两个子进程，并传入共享列表作为参数
		p1 = multiprocessing.Process(target=f, args=(l,))
		p2 = multiprocessing.Process(target=f, args=(l,))
		# 启动子进程
		p1.start()
		p2.start()
		# 等待子进程结束
		p1.join()
		p2.join()
		# 在主进程中打印共享列表的内容
		print(l)